In [ ]:
from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results

# Measurement Error Mitigation Methods
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear

# Fake Qiskit Backends
import qiskit
from qiskit.providers.fake_provider import FakeVigo, FakeTokyo

import qiskit.tools.jupyter

In [ ]:
n_qubits = 20
n_shots = 32000

backend = FakeTokyo()

# If you want to use the same total number of shots to prepare each calibration
n_shots_qiskit = n_shots / (2 ** n_qubits) # One calibration circuit per bitstring
n_shots_patch = n_shots / (2 * len(backend.configuration().coupling_map)) # 4 for each calibration, but the coupling map double counts

## Qiskit
This will take approximately forever to complete for larger devices
It will OOM if the device is large enough

In [ ]:
q_full = qiskit_full(backend, n_qubits, n_shots_qiskit)
q_lin = qiskit_linear(backend, n_qubits, n_shots_qiskit)

## Patched

In [ ]:
tpf = TensorPatchFitter(backend, n_shots=n_shots_patch)
tpf.build(verbose=True)

## The circuit to test

In [ ]:
circ = qiskit.QuantumCircuit(n_qubits, n_qubits)
initial_layout = list(range(n_qubits))

circ.h(0)
for i in range(1, n_qubits):
    circ.cnot(i - 1, i)

circ.measure(initial_layout, initial_layout)

### Circuit results

In [ ]:
tc = qiskit.transpile(circ, backend=backend, initial_layout=initial_layout, optimization_level=0)
results = qiskit.execute(tc, backend, shots=n_shots, initial_layout=initial_layout, optimization_level=0).result()
bare_res = results.get_counts()

### SIM and AIM

In [ ]:
sim_res = sim(circ, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
aim_res = aim(circ, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)

### Apply patched

In [ ]:
tpf_res= tpf.apply(bare_res)

### Apply qiskit

In [ ]:
qiskit_res_full = q_full.apply(bare_res)
qiskit_res_linear = q_lin.apply(bare_res)

### Distance Measure

In [ ]:
def dist(res, n_shots, n_qubits):
    distance = abs(res['0' * n_qubits] - n_shots / 2) + abs(res['1' * n_qubits] - n_shots / 2)
    distance += sum(abs(res[r]) for r in res if (r != '1' * n_qubits and r != '0' * n_qubits))
    distance /= n_shots
    return distance

### Results

In [ ]:
print('Bare', dist(bare_res, n_shots, n_qubits))
print('tpf', dist(tpf_res, n_shots, n_qubits))
print('sim', dist(sim_res, n_shots, n_qubits))
print('aim', dist(aim_res, n_shots // 2, n_qubits))

### Qiskit Res

In [ ]:
print('qiskit full', dist(qiskit_res_full, n_shots, n_qubits))
print('qiskit linear', dist(qiskit_res_linear, n_shots, n_qubits))

In [ ]:
print(bare_res['1' * n_qubits], bare_res['0' * n_qubits])
print(qiskit_res['1' * n_qubits], qiskit_res['0' * n_qubits])
print(tpf_res['1' * n_qubits], tpf_res['0' * n_qubits])

In [ ]:
sum(qiskit_res.values())

In [ ]:
sum(tpf_res.values())